In [1]:
import numpy as np
import keras
import keras.backend as K
import pandas as pd
import matplotlib.pyplot as plt
# import data_utils
import time
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import holidays
%matplotlib inline
%load_ext autoreload
%autoreload 2

/Users/shivanikohli/Documents/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_excel('GEFcom2014Data/GEFCom2014-E.xlsx', skiprows =range(1,17545))

In [3]:
data['month'] = np.cos(2*np.pi*data.Date.dt.month / 12)
data['dow'] = np.cos(2*np.pi*data.Date.dt.dayofweek / 7)
data['hour'] = np.cos(2*np.pi*data.Date.dt.hour/24)
us_holiday = holidays.US()
data['is_holiday'] = 1 * data.Date.dt.date.apply(lambda x: x in us_holiday)
data.drop(['Date', 'Hour'],1, inplace=True)
c = list(data.columns)
c = c[1:] + c[:1]
data = data[c]
scale = data.load.std()
offset = data.load.mean()
data.load = (data.load - offset) / scale
# data['CDD'] =  np.maximum(data.Temperature-65,0)
# data['HDD'] = np.maximum(65 - data.Temperature,0)

In [4]:
train = data[:7*len(data)//8]
val = data[3*len(data)//4:7*len(data)//8]
test = data[7*len(data)//8:]
nt = np.array(train)
nv = np.array(val)
ntt = np.array(test)
x_train = nt[:,:-1]
y_train = nt[:,-1]
x_val = ntt[:,:-1]
y_val = ntt[:,-1]

In [5]:
def row2seq_rnn(data, ys, history=24, future = 0):
    xl = []
    xff = []
    yl = []
    yf = []
    for i in np.arange(history+1, len(data)-max(future,1)):
        tmp1 = data[i-history:i+1]
        tmp2 = ys[i-history-1:i].reshape(-1,1)
        xl.append(np.append(tmp1, tmp2, axis=1))
        xff.append(data[i+1:i+future+1])
        yl.append(ys[i])
        yf.append(ys[i:i+future+1])
    return np.array(xl), np.array(xff), np.array(yl), np.array(yf)

In [6]:
xt, xtf, yt, ytf = row2seq_rnn(x_train, y_train, history=24, future=24)
xv, xvf, yv, yvf = row2seq_rnn(x_val, y_val, history=48, future = 24)
xtt, xttf, ytt, yttf = row2seq_rnn(np.array(test)[:,:-1], np.array(test)[:,-1], history=24, future=24)

In [7]:
def batch_generator(*vars, size=768):
    for i in range(0, min(len(v) for v in vars), size):
        yield (v[i:i+size] for v in vars)

In [8]:
bs = 768
n_hidden = 32

In [9]:
cond = lambda i, *_: i < 24
def pinball(y_true, y_pred):
    ts = (np.arange(1, 100, dtype=np.float32)/100).reshape(1,-1)
    pin = (tf.maximum(y_true[:,None] - y_pred, 0) * ts +
                 tf.maximum(y_pred - y_true[:,None], 0) * (1 - ts))
    return pin
def sample_body(i, s, preds):
    tmp = tf.concat((xf[:,i], preds[:,-1:]), axis=1)
    tmp.set_shape((None, xt.shape[-1]))
    out, state = lstm(tmp, s)
    new_pred = sample(tf.einsum('ij,jk->ik', out, out_weight) + out_bias)
    return tf.add(i,1), state, tf.concat((preds, new_pred), axis=1)
def predict_body(i, s, preds, loss):
    tmp = tf.concat((xf[:,i], preds[:,-1:]), axis=1)
    tmp.set_shape((None,xt.shape[-1]))
    out, state = lstm(tmp, s)
    new_pred = tf.einsum('ij,jk->ik', outputs[:,-1], out_weight) + out_bias
    intermediate_loss = tf.reduce_mean(pinball(yf[:,i], new_pred))
    return tf.add(i,1), state, tf.concat((preds, new_pred[:, 49, None]), axis=1), loss + intermediate_loss
def sample(cdfs):
    sample0 = tf.random_uniform((tf.shape(cdfs)[0],), minval=0,maxval=99,dtype=tf.int32,name=None)
    tmp = tf.transpose(tf.stack((tf.range(tf.shape(x)[0]), sample0)))
    tmp.set_shape((None,2))
    tmp = tf.gather_nd(cdfs, tmp)[:,None]
    tmp.set_shape((None, 1))
    return tmp

In [22]:
ew = lambda: tf.nn.rnn_cell.LSTMStateTuple(tf.TensorShape((None, 32)), tf.TensorShape((None, 32)))
g = tf.Graph()
with g.as_default():
    x = tf.placeholder(tf.float32, (None, None, xt.shape[2]), name='x')
    xf = tf.placeholder(tf.float32, (None, None, xtf.shape[2]), name='x_future')
    y = tf.placeholder(tf.float32, (None), name='y')
    yf = tf.placeholder(tf.float32, (None, None), name='y_future')
    learning_rate = tf.placeholder_with_default(0.001, (), name='learning_rate')
    out_weight = tf.Variable(tf.contrib.layers.xavier_initializer()((32, 99)), name='out_weight')
    out_bias= tf.Variable(tf.zeros(99), name='out_bias')

    lstm = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(32) for _ in range(2)])
    
    outputs, state = tf.nn.dynamic_rnn(lstm, x, dtype=tf.float32)
    
    for ttt in state:
        for tt in ttt:
            tt.set_shape((None, 32))
            
    i0 = tf.constant(0)
    l0 = tf.constant(0, dtype=tf.float32)
    out0 = tf.add(tf.einsum('ij,jk->ik', outputs[:,-1], out_weight), out_bias, name='out0')
    out00 = sample(out0)
    
    *_, losses = tf.while_loop(cond, predict_body, loop_vars=[i0, state, out0[:,49,None], l0],
                        shape_invariants=[i0.get_shape(), 
                                          (ew(), ew()), tf.TensorShape([None, None]), l0.get_shape()])
    _, _, predictions = tf.while_loop(cond, sample_body, loop_vars=[i0, state, out0],
                        shape_invariants=[i0.get_shape(), 
                                          (ew(), ew()), tf.TensorShape([None, None])])
    
    future_opt = tf.train.AdamOptimizer(learning_rate).minimize(tf.reduce_mean(losses))
    loss = tf.reduce_mean(pinball(y, out0))
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    saver = tf.train.Saver()

In [23]:
try:
    while True:
        tf.get_default_session().close()
except:
    pass
sess = tf.InteractiveSession(graph=g)
sess.run(tf.global_variables_initializer())

In [24]:
def train_future_batch(lr=0.001, batch_size=bs):
    perm = np.random.permutation(len(xt))
    for xs, xfs, yfs in batch_generator(xt[perm], xtf[perm], ytf[perm], size=batch_size):
        sess.run(future_opt, feed_dict={x:xs, xf: xfs, yf:yfs, learning_rate:lr})
def train_batch(lr=0.001, batch_size=bs):
    perm = np.random.permutation(len(xt))
    for xs, ys in batch_generator(xt[perm], yt[perm], size=batch_size):
        sess.run(train_op, feed_dict={x:xs, y:ys, learning_rate:lr})
def evaluate(batch_size=bs):
    ls = []
    n = 0
    perm = np.random.permutation(len(xv))
    for xs, ys in batch_generator(xv[perm], yv[perm], size=batch_size):
        ls.append(sess.run(loss, feed_dict={x:xs, y:ys})*len(xs))
        n += len(xs)
    return np.sum(ls)/n

In [25]:
bs = 768
err_list = []
for i in range(len(err_list), 10):
    start = time.time()
    train_batch(lr=0.001)
    err_list.append(evaluate())
    if i>1 and err_list[-1] < min(err_list[:-1]):
        saver.save(sess, 'GEFcom2014Models/')
    end = time.time()
    print(f'Epoch {i} ({end-start:.3f}s): loss={err_list[-1]:.5f}, best loss={min(err_list):.5f}')

Epoch 0 (10.125s): loss=0.27206, best loss=0.27206
Epoch 1 (8.315s): loss=0.19567, best loss=0.19567
Epoch 2 (13.459s): loss=0.17399, best loss=0.17399
Epoch 3 (13.965s): loss=0.26485, best loss=0.17399
Epoch 4 (10.161s): loss=0.28763, best loss=0.17399
Epoch 5 (14.821s): loss=0.27453, best loss=0.17399
Epoch 6 (12.104s): loss=0.26118, best loss=0.17399
Epoch 7 (10.356s): loss=0.25617, best loss=0.17399
Epoch 8 (10.706s): loss=0.24364, best loss=0.17399
Epoch 9 (13.939s): loss=0.22432, best loss=0.17399


In [ ]:
count_list = []
pin_list = []
bins = [-1,1,10,20,30,40,50,60,70,80,90,99,101]
for hour in range(24):
    tau = (np.arange(1, 10, dtype=np.float32)/100).reshape(1,-1)
    pin = np.mean(np.maximum(prediction[hour,:,:] - ytest[None,:,None], 0) * tau + np.maximum(ytest[None,:,None] - prediction[hour,:,:], 0) * (1 - tau), axis=(1))
    pin_list.append(pin*scale)
    counts, bins = np.histogram((prediction[hour,:,:]>ytest[None,:,None]).sum(1), bins=bins)
    count_list.append(counts)
count_array = np.array(count_list)

In [48]:
def predict(bs):
    l = []
    for xs, xfs, ys in batch_generator(xtt, xttf,  ytt, size=bs):
        l.append(sess.run(predictions, feed_dict={x:xs, xf:xfs, y:ys}))
        print(np.array(l).shape)
    return np.concatenate(l,axis = 0)

In [49]:
prediction = predict(768)

(1, 768, 123)
(2, 768, 123)
(3, 768, 123)
(4, 768, 123)
(5, 768, 123)
(6, 768, 123)
(7, 768, 123)
(8, 768, 123)
(9, 768, 123)
(10, 768, 123)
(11, 768, 123)
(12, 768, 123)
(13,)


In [33]:
prediction.shape

(9812, 123)